In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

In [2]:
demand_data = pd.read_csv('data\dataset\demand.csv')
vehicle_data = pd.read_csv('vehicle_data_edit.csv')
vehicle_fuels = pd.read_csv('data/dataset/vehicles_fuels.csv')
fuels_data = pd.read_csv('data/dataset/fuels.csv')
carbon_emissions_data = pd.read_csv('data/dataset/carbon_emissions.csv')
cost_profiles = pd.read_csv('data/dataset/cost_profiles.csv')

In [13]:
avail_vehicle = {'ID': [],
                 'distance': []}

In [14]:

buy = {}
use = {}
sell = {}

In [15]:
def feasible_vehicle(yr, s,d):
    temp_df = vehicle_data.loc[(vehicle_data['Year'] == yr) & (vehicle_data['Size'] == s)]
    if d == 'D2':
        temp_df = temp_df.loc[temp_df['Distance'] != 'D1']
    elif d == 'D3':
        temp_df = temp_df.loc[(temp_df['Distance'] == 'D3') | (temp_df['Distance'] == 'D4')]
    elif d == 'D4':
        temp_df = temp_df.loc[(temp_df['Distance'] == 'D4')]
    return temp_df

def decide_vehicle(feas_veh, prob):
    rand = random.random()
    for i in range(len(prob)):
        if rand < sum(prob[:i+1]):
            return feas_veh.iloc[i]
        
def feasible_from_avail_vehicle(avail_vehicle, feas_veh):
    new_feas_vehicle = []
    arr_distance = []
    for k in range(len(avail_vehicle['ID'])):
        i = avail_vehicle['ID'][k]
        if i in feas_veh['ID'].tolist():
            new_feas_vehicle.append(i)
            arr_distance.append(avail_vehicle['distance'][k])
    return {'ID': new_feas_vehicle, 'distance': arr_distance}

def sort_vehicle(vehicle_dict):
    list_BEV = []
    list_BEV_dist = []
    list_LNG = []
    list_LNG_dist = []
    list_diesel = []
    list_diesel_dist = []
    n = len(vehicle_dict['ID'])
    for k in range(n):
        i = vehicle_dict['ID'][0]
        temp = i.split('_')
        if temp[0] == 'BEV':
            list_BEV.append(i)
            list_BEV_dist.append(vehicle_dict['distance'][0])
        elif temp[0] == 'LNG':
            list_LNG.append(i)
            list_LNG_dist.append(vehicle_dict['distance'][0])
        else:
            list_diesel.append(i)
            list_diesel_dist.append(vehicle_dict['distance'][0])
        del vehicle_dict['ID'][0]
        del vehicle_dict['distance'][0]

    return {'ID': list_BEV + list_LNG + list_diesel, 'distance': list_BEV_dist + list_LNG_dist + list_diesel_dist}

# def carbon_capture():

In [16]:
unique_year = np.unique(demand_data['Year'])
s = np.unique(demand_data['Size'])
d = np.unique(demand_data['Distance'])

In [17]:
#initial
year = 2023
init_demand = demand_data.loc[demand_data['Year'] == year]
buy[year] = {'ID': [],
             'distance': []}
demand_avail_dict = {}
demand_dict = {}
for i in range(len(s)):
    demand_dict[s[i]] = {}
    demand_avail_dict[s[i]] = {}
    demand_avail = 0
    for j in range(len(d)-1,-1,-1):
        demand = init_demand['Demand (km)'].loc[(init_demand['Size'] == s[i]) & (init_demand['Distance'] == d[j])].iloc[0]
        demand_dict[s[i]][d[j]] = demand
        feas_veh = feasible_vehicle(year,s[i],d[j])
        yearly_cost = 0.12*feas_veh['Cost ($)'] + feas_veh['fuel_cost']
        feas_veh['yearly_cost'] = yearly_cost
        inv = 1/feas_veh['yearly_cost']
        inv[inv.idxmax()] = inv[inv.idxmax()]*4
        prob = (inv/sum(inv)).values
        while demand_avail < demand:
            temp_df_decide = decide_vehicle(feas_veh, prob)
            buy[year]['ID'].append(temp_df_decide['ID'])
            buy[year]['distance'].append(temp_df_decide['Yearly range (km)'])
            demand_avail = demand_avail + temp_df_decide['Yearly range (km)']
        demand_avail_dict[s[i]][d[j]] = demand_avail
        demand_avail = demand_avail - demand

C:\Users\asus\AppData\Local\Temp\ipykernel_13040\1389255454.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feas_veh['yearly_cost'] = yearly_cost
C:\Users\asus\AppData\Local\Temp\ipykernel_13040\1389255454.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feas_veh['yearly_cost'] = yearly_cost
C:\Users\asus\AppData\Local\Temp\ipykernel_13040\1389255454.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

In [18]:
demand_dict

{'S1': {'D4': 414315, 'D3': 3292011, 'D2': 2597094, 'D1': 869181},
 'S2': {'D4': 133677, 'D3': 778008, 'D2': 1383196, 'D1': 995694},
 'S3': {'D4': 205426, 'D3': 1002466, 'D2': 2431901, 'D1': 2183475},
 'S4': {'D4': 1809, 'D3': 118899, 'D2': 754717, 'D1': 14576}}

In [19]:
avail_vehicle['ID'] = avail_vehicle['ID'] + buy[2023]['ID']

In [20]:
avail_vehicle['distance'] = avail_vehicle['distance'] + buy[2023]['distance']

In [22]:
use[2023] = {'ID': [],
             'size': [],
             'distance_bucket': [],
             'distance': []
             }
for i in range(len(s)):
    for j in range(len(d)-1,-1,-1):
        demand = init_demand['Demand (km)'].loc[(init_demand['Size'] == s[i]) & (init_demand['Distance'] == d[j])].iloc[0]
        feas_veh = feasible_vehicle(year,s[i],d[j])
        demand_satisfied = 0
        while demand - demand_satisfied != 0:
            new_feas_vehicle = feasible_from_avail_vehicle(avail_vehicle,feas_veh)
            sorted_feas_vehicle = sort_vehicle(new_feas_vehicle)
            veh = sorted_feas_vehicle['ID'][0]
            temp_df = feas_veh.loc[feas_veh['ID'] == veh]
            dist = min([demand - demand_satisfied,sorted_feas_vehicle['distance'][0]])
            demand_satisfied = demand_satisfied + dist
            use[year]['ID'].append(veh)
            use[year]['size'].append(s[i])
            use[year]['distance_bucket'].append(d[j])
            use[year]['distance'].append(dist)

            idx = avail_vehicle['ID'].index(veh)
            temp_list = [0 for i in range(len(avail_vehicle['distance']))]
            temp_list[idx] = dist

            avail_vehicle['distance'] = np.array(avail_vehicle['distance']) - np.array(temp_list)
            is_zero = np.argwhere(abs(avail_vehicle['distance']) <= 0.01).tolist()
            if len(is_zero) == 0:
                avail_vehicle['distance'] = avail_vehicle['distance'].tolist()
            else:
                avail_vehicle['distance'] = avail_vehicle['distance'].tolist()
                del avail_vehicle['ID'][is_zero[0][0]]
                del avail_vehicle['distance'][is_zero[0][0]]

In [27]:
df_use = pd.DataFrame(use[2023])

In [31]:
sum(df_use.loc[(df_use['size'] == 'S3') & (df_use['distance_bucket'] == 'D1')]['distance'])

2183475

In [134]:
len(avail_vehicle['ID'])

186

In [133]:
feas_veh

,Unnamed: 0,ID,Vehicle,Size,Year,Cost ($),Yearly range (km),Distance,fuel,fuel_cost,total_cost
0,0,BEV_S1_2023,BEV,S1,2023,187000,102000,D1,Electricity,13088.52137,31788.52137
64,64,Diesel_S1_2023,Diesel,S1,2023,85000,102000,D4,B20,32671.05472,41171.05472
128,128,LNG_S1_2023,LNG,S1,2023,100000,102000,D4,LNG,16349.55384,26349.55384


In [86]:
feas_veh['ID'].iloc[:]

32        BEV_S4_2023
96     Diesel_S4_2023
160       LNG_S4_2023
Name: ID, dtype: object